In [1]:
import cv2 as cv
import numpy as np
import math

backsub = cv.createBackgroundSubtractorKNN()
#capture = cv.VideoCapture(0)  # for mac real time webcam
capture = cv.VideoCapture("/Users/istiakahmedseyam/PyCharmMiscProject/sample 2.mp4")
# capture = cv.VideoCapture(0, cv.CAP_DSHOW)  # for windows real time webcam



if not capture.isOpened():
    print('Unable to open')
    exit(0)

fps = capture.get(cv.CAP_PROP_FPS)
if fps == 0:
    fps = 30


object_centers = {}
object_speeds = {}
object_id_count = 0


PIXEL_TO_METER = 0.05
SPEED_LIMIT = 50  # km/h

while True:
    ret, frame = capture.read()
    if not ret:
        break

    frame = cv.resize(frame, (800, 800))
    fgMask = backsub.apply(frame)

    kernel = np.ones((5, 5), np.uint8)
    fgMask = cv.erode(fgMask, kernel, iterations=2)
    fgMask = cv.dilate(fgMask, kernel, iterations=2)

    # Clean mask
    fgMask[np.abs(fgMask) < 250] = 0

    contours, _ = cv.findContours(fgMask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    new_centers = {}

    for cnt in contours:
        area = cv.contourArea(cnt)
        if area < 800:
            continue

        x, y, w, h = cv.boundingRect(cnt)
        cx = x + w // 2
        cy = y + h // 2

        matched_id = None

        # Match with previous objects
        for obj_id, (px, py) in object_centers.items():
            dist = math.hypot(cx - px, cy - py)
            if dist < 50:
                matched_id = obj_id
                break

        if matched_id is None:
            matched_id = object_id_count
            object_id_count += 1

        new_centers[matched_id] = (cx, cy)

        # Speed calculation
        if matched_id in object_centers:
            px, py = object_centers[matched_id]
            pixel_dist = math.hypot(cx - px, cy - py)

            speed_mps = pixel_dist * PIXEL_TO_METER * fps
            speed_kmh = speed_mps * 3.6

            object_speeds[matched_id] = speed_kmh
        else:
            speed_kmh = 0

        speed = object_speeds.get(matched_id, 0)

        # Color rule
        if speed > SPEED_LIMIT:
            color = (0, 0, 255)  # red
            label = f"{int(speed)} km/h"
        else:
            color = (0, 255, 0)  # green
            label = f"{int(speed)} km/h"

        cv.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv.putText(frame, label, (x, y - 10),
                   cv.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    object_centers = new_centers

    cv.rectangle(frame, (10, 2), (120, 25), (255, 255, 255), -1)
    cv.putText(frame, str(int(capture.get(cv.CAP_PROP_POS_FRAMES))),
               (15, 18), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0))

    cv.imshow('Frame', frame)
    cv.imshow('FG Mask', fgMask)

    keyboard = cv.waitKey(30)
    if keyboard == ord('q') or keyboard == 27:
        break

capture.release()
cv.destroyAllWindows()


KeyboardInterrupt: 

In [ ]:
quit